# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [59]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [60]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [61]:
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [62]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [63]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [64]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [65]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Query 1 
 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

-  table song_detail_by_session has sessionid and itemInSession  used as a partition key because the queries will filter by this column's.

In [66]:
query = "CREATE TABLE IF NOT EXISTS song_detail_by_session "
query = query + """
                (sessionId int, 
                itemInSession int ,
                artist text,
                song text,
                length float, 
                PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)               

In [67]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_detail_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [68]:
query = "select artist, song, length from song_detail_by_session WHERE sessionId=338 and itemInSession=4; "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- For table artist_song_listeing_by_userid_session the (userid & sessionId) was used as a partition key because the queries will filter by this column's.
<br> and itemInSession to sort the data 

In [69]:
query = "CREATE TABLE IF NOT EXISTS artist_song_listeing_by_userid_session "
query = query + """
                (userid int, 
                sessionId int,
                itemInSession int,
                firstName text,
                lastName text,
                artist_name text,
                song text, 
                PRIMARY KEY ((userid, sessionId), itemInSession, firstName, lastName))"""
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [70]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_listeing_by_userid_session (userid, sessionId, itemInSession, firstName, lastName, artist_name, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[1],line[4],line[1]+" "+line[4],line[9]))

In [71]:
query = "select artist_name, song from artist_song_listeing_by_userid_session WHERE userid = 10 and sessionid = 182; "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist_name, row.song)
                    

Sylvie Cruz Keep On Keepin' On
Sylvie Cruz Greece 2000
Sylvie Cruz Kilometer
Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


# Query 3
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- For table user_name_by_song the  song was used as a partition key because the queries will filter by this column's.
<br> userid were used as clustering columns to help make up a unique key.

In [72]:
query = "CREATE TABLE IF NOT EXISTS user_name_by_song "
query = query + """
                (song text,
                userid int,
                firstName text,
                lastName text,
                PRIMARY KEY (song, userid))"""
try:
    session.execute(query)
except Exception as e:
    print(e)     

In [73]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_name_by_song (song, userid, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [74]:
query = "select firstName, lastName from user_name_by_song WHERE song='All Hands Against His Own'; "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [75]:
query = "drop table song_detail_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [76]:
query = "drop table artist_song_listeing_by_userid_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [77]:
query = "drop table user_name_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [78]:
session.shutdown()
cluster.shutdown()